In [13]:
# librerias
import pandas as pd
import numpy as np

In [2]:
import joblib

def load_df(path):
    df = joblib.load(path)
    return df

In [8]:
df = load_df('../tmp/fe_df.pkl')

In [9]:
df

,fisica,virtual,no_prime,si_prime,f,m,no_def,NA_establecimiento,abarrotes,farmacia,...,dcto,cashback,linea_tc,interes_tc,sin_dia,cos_dia,sin_hora,cos_hora,fraude,fecha
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.263136,-0.176315,0.389427,0.289465,-9.510565e-01,-0.309017,-0.866025,0.500000,0,2020-01-21
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,-1.006380,0.389427,0.289465,5.665539e-16,-1.000000,0.965926,-0.258819,0,2020-01-15
2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,2.139704,1.440304,0.811001,-4.067366e-01,-0.913545,0.258819,-0.965926,0,2020-01-17
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.508714,0.316006,1.440304,0.811001,8.660254e-01,0.500000,0.258819,-0.965926,0,2020-01-05
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,-0.834270,1.440304,0.811001,-1.133108e-15,1.000000,0.500000,-0.866025,0,2020-01-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26970,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,0.269322,-1.026972,0.811001,9.945219e-01,-0.104528,-0.258819,-0.965926,0,2020-01-08
26971,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,0.708078,-1.026972,0.811001,2.079117e-01,-0.978148,-0.707107,-0.707107,0,2020-01-14
26972,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,-0.277543,-1.026972,0.811001,-2.079117e-01,-0.978148,0.258819,-0.965926,0,2020-01-16
26973,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.438280,-1.355553,-1.026972,0.811001,4.067366e-01,-0.913545,-0.965926,-0.258819,0,2020-01-13


# Train and test data

In [106]:
df['fraude'].value_counts()

0    26165
1      810
Name: fraude, dtype: int64

In [107]:
df_no_fraude = df[df['fraude'] == 0]
df_si_fraude = df[df['fraude'] == 1]

In [108]:
print(df_no_fraude.shape)
print(df_si_fraude.shape)

(26165, 29)
(810, 29)


In [109]:
810*.7

567.0

In [118]:
810*.29

234.89999999999998

In [119]:
810*.01

8.1

In [174]:
235+567

802

In [175]:
muestra_no_fraude = df_no_fraude.sample(n=800,random_state=12)

In [176]:
muestra_no_fraude

,fisica,virtual,no_prime,si_prime,f,m,no_def,NA_establecimiento,abarrotes,farmacia,...,dcto,cashback,linea_tc,interes_tc,sin_dia,cos_dia,sin_hora,cos_hora,fraude,fecha
10478,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-0.508714,-1.065944,1.668755,-1.588062,-0.951057,0.309017,0.707107,0.707107,0,2020-01-24
13944,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.426779,-0.107605,1.531684,-0.023456,0.866025,0.500000,0.707107,0.707107,0,2020-01-05
13540,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,1.222416,1.668755,0.706693,0.406737,-0.913545,0.866025,0.500000,0,2020-01-13
18068,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.508714,2.130545,-0.204547,-0.440684,-0.994522,0.104528,-0.500000,0.866025,0,2020-01-23
24214,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.508714,0.073498,1.394613,0.080851,-0.994522,0.104528,-0.258819,-0.965926,0,2020-01-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13894,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,-0.508714,-1.122498,-0.295927,1.228229,0.743145,-0.669131,-0.258819,0.965926,0,2020-01-11
16892,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,-0.038904,-0.524379,-0.857913,0.743145,-0.669131,0.500000,0.866025,0,2020-01-11
6381,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.508714,-1.323588,1.440304,-0.127763,0.406737,-0.913545,-0.500000,0.866025,0,2020-01-13
16450,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.508714,1.611241,0.206666,-0.649298,-0.207912,0.978148,0.258819,-0.965926,0,2020-01-29


In [177]:
muestra_si_fraude = df_si_fraude.sample(n=800,random_state=12)

In [178]:
muestra_si_fraude

,fisica,virtual,no_prime,si_prime,f,m,no_def,NA_establecimiento,abarrotes,farmacia,...,dcto,cashback,linea_tc,interes_tc,sin_dia,cos_dia,sin_hora,cos_hora,fraude,fecha
8397,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.479646,-0.179378,0.023905,-1.379448,7.431448e-01,-0.669131,-0.707107,7.071068e-01,1,2020-01-11
12446,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.728415,-1.023284,0.937710,1.019615,-1.133108e-15,1.000000,0.707107,-7.071068e-01,1,2020-01-30
17488,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,-0.508714,-0.820087,0.846330,-0.336377,-9.945219e-01,0.104528,-0.707107,7.071068e-01,1,2020-01-23
12795,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,0.080841,-1.072662,1.541150,-5.877853e-01,-0.809017,-0.500000,8.660254e-01,1,2020-01-18
3760,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,-0.533426,1.120472,0.811001,-8.660254e-01,0.500000,-0.866025,-5.000000e-01,1,2020-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24494,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,-0.211792,0.572188,-1.483755,7.431448e-01,0.669131,0.707107,7.071068e-01,1,2020-01-04
2685,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.508714,-1.028798,-0.021786,0.289465,-9.510565e-01,0.309017,-0.866025,-5.000000e-01,1,2020-01-24
12445,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,-0.523054,0.937710,1.019615,-5.877853e-01,-0.809017,0.866025,5.000000e-01,1,2020-01-18
24756,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.508714,0.401065,1.348923,-0.336377,-4.067366e-01,-0.913545,1.000000,6.123234e-17,1,2020-01-17


In [179]:
df_nuevo = pd.concat([muestra_si_fraude,muestra_no_fraude],axis=0)

In [180]:
df_nuevo['fraude'].value_counts()

0    800
1    800
Name: fraude, dtype: int64

In [181]:
X = df_nuevo.drop(columns=['fraude','fecha'], axis=1).values
X

array([[ 0.        ,  1.        ,  1.        , ..., -0.66913061,
        -0.70710678,  0.70710678],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.70710678, -0.70710678],
       [ 1.        ,  0.        ,  1.        , ...,  0.10452846,
        -0.70710678,  0.70710678],
       ...,
       [ 1.        ,  0.        ,  1.        , ..., -0.91354546,
        -0.5       ,  0.8660254 ],
       [ 0.        ,  1.        ,  1.        , ...,  0.9781476 ,
         0.25881905, -0.96592583],
       [ 0.        ,  1.        ,  1.        , ...,  0.80901699,
        -0.5       ,  0.8660254 ]])

In [182]:
y = df_nuevo['fraude']
y

8397     1
12446    1
17488    1
12795    1
3760     1
        ..
13894    0
16892    0
6381     0
16450    0
2903     0
Name: fraude, Length: 1600, dtype: int64

In [183]:
from sklearn.model_selection import train_test_split

In [184]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, stratify=y, random_state=12)

In [185]:
print(X.shape, X_train.shape, X_test.shape)

(1600, 27) (1280, 27) (320, 27)


# Modelo

In [186]:
model = LogisticRegression()

In [187]:
model.fit(X_train, y_train)

LogisticRegression()

In [188]:
X_train_prediction = model.predict(X_train)

In [200]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [190]:
data_train_precision = precision_score(X_train_prediction, y_train)

In [198]:
data_train_accuracy = accuracy_score(X_train_prediction, y_train)

In [203]:
data_train_recall = recall_score(X_train_prediction, y_train)

In [204]:
data_train_accuracy

0.5453125

In [205]:
data_train_precision

0.5375

In [206]:
data_train_recall

0.546031746031746

In [192]:
X_train_prediction

array([1, 0, 1, ..., 0, 1, 1])

In [ ]:
X_train_prediction = model.predict(X_train)